# Developing an LLM application using Langchain

In [ ]:
# Run this cell so you do not see any warnings
import warnings
warnings.filterwarnings('ignore')

## Task 1: Import Libraries

In [ ]:
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import textwrap
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

genai.configure(api_key='YOUR API KEY')
model = genai.GenerativeModel('gemini-2.5-flash')

## Task 2: Ask The Questions Using Prompts

In [ ]:
response = model.generate_content("Explain Generative AI with 3 bullet points")
print(response.text)
Markdown(response.text)

## Task 3: Chat With Gemini And Retrieve The Chat History

In [ ]:
hist = model.start_chat()
response = hist.send_message("Hi! Give me a recipe to make a margeritta pizza from scratch.")
Markdown(response.text)

for i in hist.history:
    print(i)
    print('\n\n')
i.parts[0].text 

model.count_tokens("Now please help me find the nearest supermarket from where I can buy the ingredients.")

## Task 4: Experiment With The Temperature Parameter

In [ ]:
def get_response(prompt, generation_config={}):
    response = model.generate_content(contents=prompt, 
    generation_config=generation_config)
    return response

for temp in [0.0, 0.25, 0.5, 0.75, 1.0]:
  config = genai.types.GenerationConfig(temperature=temp)
  result = get_response("Explain the concepts of XGBoost and Random Forest with real-life use cases", generation_config=config)

  print(f"\n\nFor temperature value {temp}, the results are: \n\n")
  display(Markdown(result.text))

## Task 5: Experiment With Maximum Output Tokens

In [ ]:
def get_response(prompt, generation_config={}):
    response = model.generate_content(contents=prompt, generation_config=generation_config)
    return response

for m_o_tok in [1000, 2000, 7000]:
    config = genai.types.GenerationConfig(max_output_tokens=m_o_tok)
    
    print(f"\n\n--- Max Output Tokens: {m_o_tok} ---")
    try:
        # Get the result
        result = get_response("Explain the concepts of XGBoost and Random Forest with real-life use cases", generation_config=config)
        
        # Only print text if the model actually returned text parts
        if result.candidates[0].content.parts:
            display(Markdown(result.text))
        # Handle the empty response
        # This is because different models have different MOTok to even start generating the answer
        else: 
            print(f"Result was empty! The model stopped because: {result.candidates[0].finish_reason}")
            print("(Reason 2 means 'MAX_TOKENS' was reached before a complete word could be formed)")
            
    except Exception as e:
        print(f"Skipped due to error: {e}")

## Task 6: Experiment With the top_k Parameter

In [ ]:
def get_response(prompt, generation_config={}):
    response = model.generate_content(contents=prompt, 
    generation_config=generation_config)
    return response

for k in [1, 4, 16, 32, 40]:
    config = genai.types.GenerationConfig(top_k=k)
    result = get_response("Explain the concepts of XGBoost and Random Forest with real-life use cases", generation_config=config)

    print(f"\n\nFor top k value {k}, the results are: \n\n")
    display(Markdown(result.text))

## Task 7: Experiment With the top_p Parameter

In [ ]:
def get_response(prompt, generation_config={}):
    response = model.generate_content(contents=prompt, 
    generation_config=generation_config)
    return response

for p in [0, 0.2, 0.4, 0.8, 1]:
    config = genai.types.GenerationConfig(top_p=p)
    result = get_response("Explain the concepts of XGBoost and Random Forest with real-life use cases", generation_config=config)

    print(f"\n\nFor top p value {p}, the results are: \n\n")
    display(Markdown(result.text))

## Task 8: Experiment With the candidate_count Parameter

In [ ]:
config = genai.types.GenerationConfig(candidate_count=1)
result = get_response("Explain the concepts of XGBoost and Random Forest with real-life use cases", generation_config=config)
Markdown(result.text)

## Task 9: Introduction to Retrieval Augmented Generation

## Task 10: Load the PDF and Extract the Texts

In [ ]:
CHUNK_SIZE = 700
CHUNK_OVERLAP = 100
pdf_path = "http://jmc.stanford.edu/articles/whatisai/whatisai.pdf"

In [ ]:
pdf_loader = PyPDFLoader(pdf_path)
split_pdf_document = pdf_loader.load_and_split()

In [ ]:
# Splitting text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
context = "\n\n".join(str(p.page_content) for p in split_pdf_document)
texts = text_splitter.split_text(context)

## Task 11: Create the Gemini Model and Create the Embeddings

In [ ]:
gemini_model = ChatGoogleGenerativeAI(model='gemini-2.5-flash', google_api_key= "YOUR API KEY" , temperature=0.8)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key= "YOUR API KEY" )

In [ ]:
vector_index = Chroma.from_texts(texts, embeddings)
retriever = vector_index.as_retriever(search_kwargs={"k" : 5})

## Task 12: Create the RAG Chain and Ask Query

In [ ]:
qa_chain = RetrievalQA.from_chain_type(gemini_model, retriever=retriever, return_source_documents=True)

In [ ]:
# Example usage 
question = "Who can learn AI?"
result = qa_chain.invoke({"query": question})
print("Answer:", result["result"])